In [131]:
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen
import pandas as pd
import folium

부산 버스 모든 노선의 ID를 구해서 xlsx파일로 저장하기.

In [ ]:
bus_all_line = []
def busan_bus_line():
    url = 'http://apis.data.go.kr/6260000/BusanBIMS/busInfo'
    params ={'serviceKey' : 'l2SER5UdFnVMTCfW/75Op1w1xj0ZVJyWjxZCA4s2qcMkJIjXonsQeorj8k35mmylOV0ywqc75RoXXtagwXuUnQ=='}

    response = requests.get(url, params=params)
    decodecontent = response.content.decode('utf-8') 
    xmlParser = BeautifulSoup(decodecontent,'xml')
    contents =  xmlParser.select('item')
    for number in range(0,len(contents)):
        content = contents[number].text.split('\n')
        content.remove('')
        content.remove('')
        bus_all_line.append(content)


busan_bus_line()


In [14]:
df_bus_line = pd.DataFrame(bus_all_line)

In [18]:
df_bus_line.columns = ['노선ID','버스 번호','버스 종류','기점명','종점명','버스 회사명','배차간격','첫차','막차','배차간격(출퇴근)',
                       '배차간격(평일)','배차간격(휴일)']

In [19]:
df_bus_line.to_excel('./data/부산버스라인.xlsx',index=False)

부산의 버스 라인으로 검색하면 노선의 ID로 바꾸어 검색하기

In [21]:
bus_id_num = pd.read_excel('./data/부산버스라인.xlsx',usecols=['노선ID','버스 번호'])

In [75]:
# 버스 번호에 해당하는 노선 ID 찾기
def bus(num):
    condition = bus_id_num['버스 번호'] == str(num)
    index = bus_id_num[condition].index
    return bus_id_num.values[index][0][0]
    

print(bus('1002(심야)'))

5201002F00


<?xml version="1.0" encoding="utf-8"?>
<response>
<header>
<resultCode>00</resultCode>
<resultMsg>NORMAL SERVICE</resultMsg>
</header>
<body>
<items>
<item>
<bstopidx>1</bstopidx>
<bstopnm>금정공영차고지</bstopnm>
<nodeid>217210101</nodeid>
<lineno>80</lineno>
<rpoint>0</rpoint>
</item>
<item>
<bstopidx>2</bstopidx>
<bstopnm>부산종합터미널.노포역</bstopnm>
<nodeid>209010201</nodeid>
<lineno>80</lineno>
<arsno>11006</arsno>
<avgym>180</avgym>
<rpoint>0</rpoint>
</item>
<item>
<bstopidx>3</bstopidx>
<bstopnm>노포삼거리</bstopnm>
<nodeid>210760301</nodeid>
<lineno>80</lineno>
<direction>1</direction>
<gpsym>212434</gpsym>
<carno>70자3157</carno>
<lat>35.2798816</lat>
<lin>129.0932608</lin>
<nodekn>0</nodekn>
<arsno>11009</arsno>
<avgym>87</avgym>
<rpoint>0</rpoint>
<lowplate>0</lowplate>
</item>
<item>
<bstopidx>4</bstopidx>
<bstopnm>범어사입구</bstopnm>
<nodeid>175290301</nodeid>
<lineno>80</lineno>
<arsno>11012</arsno>
<avgym>142</avgym>
<rpoint>0</rpoint>
</item>
<item>
<bstopidx>5</bstopidx>
<bstopnm>어린이놀이터</bst

In [183]:
bus_now_list= []
def bus_now(bus_num):
    condition = bus_id_num['버스 번호'] == str(bus_num)
    index = bus_id_num[condition].index
    lineid = bus_id_num.values[index][0][0]
    url = 'http://apis.data.go.kr/6260000/BusanBIMS/busInfoByRouteId'
    params ={'serviceKey' : 'l2SER5UdFnVMTCfW/75Op1w1xj0ZVJyWjxZCA4s2qcMkJIjXonsQeorj8k35mmylOV0ywqc75RoXXtagwXuUnQ==',
             'lineid' : lineid}
    response = requests.get(url, params=params)
    decodecontent = response.content.decode('utf-8') 
    xmlParser = BeautifulSoup(decodecontent,'xml')
    contents = xmlParser.select('item')
    for number in range(0,len(contents)):
        content = contents[number].text.split('\n')
        content.remove('')
        content.remove('')
        bus_now_list.append(content)
    return bus_now_list

In [186]:
exam = bus_now(80)
exam

[['1', '금정공영차고지', '217210101', '80', '0'],
 ['2', '부산종합터미널.노포역', '209010201', '80', '11006', '180', '0'],
 ['3', '노포삼거리', '210760301', '80', '11009', '87', '0'],
 ['4', '범어사입구', '175290301', '80', '11012', '142', '0'],
 ['5', '어린이놀이터', '174770301', '80', '11015', '53', '0'],
 ['6', '한아름빌라', '173830202', '80', '11017', '54', '0'],
 ['7', '남산동.요산문학관', '173830201', '80', '11018', '54', '0'],
 ['8', '금샘초등학교', '173830203', '80', '11027', '53', '0'],
 ['9', '남산고교', '174890201', '80', '11028', '55', '0'],
 ['10',
  '부산외국어대학교',
  '174890202',
  '80',
  '1',
  '215225',
  '70자3216',
  '35.2656608',
  '129.0852352',
  '0',
  '11036',
  '57',
  '0',
  '1'],
 ['11', '남산성당', '209660201', '80', '11162', '36', '0'],
 ['12', '남산119안전센터', '506050000', '80', '11286', '62', '0'],
 ['13', '샘골공원', '176700101', '80', '11039', '41', '0'],
 ['14', '선경3차아파트', '174860301', '80', '11047', '64', '0'],
 ['15', '신동아아파트', '175210201', '80', '11048', '70', '0'],
 ['16', '롯데캐슬정문', '174910101', '80', '11053', '71', '0'

In [132]:
stop_general = pd.read_excel('./data/부산버스_일반.xlsx')
location = stop_general[['GPS X 좌표','GPS Y 좌표']]

In [190]:
map =folium.Map(location=[location['GPS Y 좌표'].mean(),location['GPS X 좌표'].mean()],zoom_start=12)


for i in range(0,len(exam)):
    if exam[i][4] == '1':
        try:
            folium.Marker(location=[exam[i][7],exam[i][8]],
                        icon=folium.Icon(color='blue')).add_to(map)
        except:
            pass
    else:
        try:
            folium.Marker(location=[exam[i][7],exam[i][8]],
                        icon=folium.Icon(color='red')).add_to(map)
        except:
            pass
map.save('./data/부산버스 현재 위치.html')